# Script to load CSV data into RDS database using SQLAlchemy models.

## This script handles the ETL process for loading rental marketplace data from CSV file into a MYSQL RDS database. It processes four main entities:
1. Apartments - Core property listings
2. Apartment Attributes - Detailed property characteristics
3. User Viewings - User interaction tracking
4. Bookings - Reservation transactions

## Import necessary libraries

In [31]:
import pandas as pd
import os
import sys
from pathlib import Path
from sqlalchemy import create_engine

# Add project root to Python path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

## Get the correct data source

In [6]:
# Get the current working directory and navigate to project root
current_dir = Path.cwd()
project_root = current_dir.parent
data_dir = project_root / "data"

print(f"Project root directory: {project_root}")
print(f"Data directory: {data_dir}")


Project root directory: c:\DOJO\VSCode Project\Notebooks\Labs\Phase_II_labs\3_rental_marketplace
Data directory: c:\DOJO\VSCode Project\Notebooks\Labs\Phase_II_labs\3_rental_marketplace\data


## Load data into a pandas DataFrame

In [ ]:
apt_att_file = os.path.join(data_dir, "apartment_attributes.csv")
apt_file = os.path.join(data_dir, "apartments.csv")
bookings_file = os.path.join(data_dir, "bookings.csv")
user_viewing_file = os.path.join(data_dir, "user_viewing.csv")

# Read the CSV files into DataFrames
apt_att = pd.read_csv(apt_att_file)
apt = pd.read_csv(apt_file)
bookings = pd.read_csv(bookings_file)
user_viewing = pd.read_csv(user_viewing_file)

# Display the first few rows of each DataFrame to verify the data

Apartment Attributes DataFrame:
    id   category                                               body  \
0   1       2BHK  Happy product model process necessary. Only fo...   
1   2  Penthouse  Technology past much. Shoulder collection appr...   
2   3       3BHK  Firm agreement shake design sort. Size source ...   
3   4       1BHK  Next may hear camera. Heart dinner onto increa...   
4   5       1BHK  Space necessary each statement sport early. Fi...   

                       amenities  bathrooms  bedrooms     fee  has_photo  \
0      Balcony, Air Conditioning          1         1  168.84       True   
1  Garden, Pet-friendly, Balcony          2         2  202.99      False   
2                            Gym          2         1  488.93      False   
3                         Garden          3         3  376.10       True   
4                   Pet-friendly          2         2  465.79      False   

  pets_allowed price_display price_type  square_feet  \
0        False      $1588.4

## Verify the data

In [15]:
apt_att.info()
# apt_att.describe()
apt_att.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             200000 non-null  int64  
 1   category       200000 non-null  object 
 2   body           200000 non-null  object 
 3   amenities      200000 non-null  object 
 4   bathrooms      200000 non-null  int64  
 5   bedrooms       200000 non-null  int64  
 6   fee            200000 non-null  float64
 7   has_photo      200000 non-null  bool   
 8   pets_allowed   70956 non-null   object 
 9   price_display  200000 non-null  object 
 10  price_type     200000 non-null  object 
 11  square_feet    200000 non-null  int64  
 12  address        200000 non-null  object 
 13  cityname       200000 non-null  object 
 14  state          200000 non-null  object 
 15  latitude       200000 non-null  float64
 16  longitude      200000 non-null  float64
dtypes: bool(1), float64(3), int64

,id,category,body,amenities,bathrooms,bedrooms,fee,has_photo,pets_allowed,price_display,price_type,square_feet,address,cityname,state,latitude,longitude
0,1,2BHK,Happy product model process necessary. Only fo...,"Balcony, Air Conditioning",1,1,168.84,True,False,$1588.42,Monthly,1463,"92525 Holt Turnpike Lake Keith, KY 50153",New York,California,20.457092,0.466220
1,2,Penthouse,Technology past much. Shoulder collection appr...,"Garden, Pet-friendly, Balcony",2,2,202.99,False,True,$3585.32,Yearly,722,"89610 Chang Lane Apt. 295 New Ericaland, NC 28192",Austin,Texas,38.182993,-129.769256
2,3,3BHK,Firm agreement shake design sort. Size source ...,Gym,2,1,488.93,False,False,$1961.69,Monthly,2208,"134 Bryan Island Leefort, VT 48142",Chicago,Illinois,61.585391,-92.597924
3,4,1BHK,Next may hear camera. Heart dinner onto increa...,Garden,3,3,376.10,True,False,$4845.75,Yearly,589,"13660 Amanda Isle East Michaelchester, CT 62513",San Antonio,Texas,-59.015450,6.296552
4,5,1BHK,Space necessary each statement sport early. Fi...,Pet-friendly,2,2,465.79,False,True,$2599.65,One-time,1590,"7915 Laurie Manor Port John, MN 73309",Los Angeles,California,-48.009442,-33.172952


In [16]:
apt.info()
apt.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       200000 non-null  int64  
 1   title                    200000 non-null  object 
 2   source                   200000 non-null  object 
 3   price                    200000 non-null  float64
 4   currency                 200000 non-null  object 
 5   listing_created_on       200000 non-null  object 
 6   is_active                200000 non-null  bool   
 7   last_modified_timestamp  200000 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(5)
memory usage: 10.9+ MB


,id,title,source,price,currency,listing_created_on,is_active,last_modified_timestamp
0,1,"Johnson, Fowler and Johnson",Realtor,4020.95,EUR,27/03/2024,True,27/12/2021
1,2,Kelley-Johnson,Craigslist,1390.01,INR,23/07/2023,True,10/02/2023
2,3,"Gibson, Delgado and Austin",Craigslist,3082.67,INR,15/04/2024,True,02/01/2020
3,4,"Kelly, Smith and Gibson",Zillow,4724.20,USD,27/10/2020,False,23/06/2023
4,5,Carroll-Burch,Airbnb,3867.11,USD,01/02/2021,False,25/07/2022


In [17]:
bookings.info()
bookings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   booking_id      500000 non-null  int64  
 1   user_id         500000 non-null  int64  
 2   apartment_id    500000 non-null  int64  
 3   booking_date    500000 non-null  object 
 4   checkin_date    500000 non-null  object 
 5   checkout_date   500000 non-null  object 
 6   total_price     500000 non-null  float64
 7   currency        500000 non-null  object 
 8   booking_status  500000 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 34.3+ MB


,booking_id,user_id,apartment_id,booking_date,checkin_date,checkout_date,total_price,currency,booking_status
0,1,2920,130940,11/11/2022,07/09/2024,30/06/2023,2923.67,EUR,confirmed
1,2,2788,191497,17/05/2024,07/07/2024,06/09/2021,4645.29,EUR,pending
2,3,9285,139117,26/11/2022,12/02/2025,07/04/2023,1683.99,INR,canceled
3,4,8161,118131,09/07/2020,25/10/2024,08/02/2021,1870.62,USD,confirmed
4,5,6733,43470,08/05/2024,31/01/2022,29/10/2024,3379.44,INR,pending


In [18]:
user_viewing.info()
user_viewing.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   user_id         300000 non-null  int64 
 1   apartment_id    300000 non-null  int64 
 2   viewed_at       300000 non-null  object
 3   is_wishlisted   300000 non-null  bool  
 4   call_to_action  300000 non-null  object
dtypes: bool(1), int64(2), object(2)
memory usage: 9.4+ MB


,user_id,apartment_id,viewed_at,is_wishlisted,call_to_action
0,5353,180678,23/01/2023,False,Reported
1,8341,12140,14/05/2021,True,Reported
2,5517,72896,27/06/2023,True,Contact Agent
3,1710,19727,15/10/2022,True,Shortlisted
4,896,44550,06/02/2020,False,Reported


## Set connection to MYSQL database

In [ ]:
from src.config.settings import DB_CONFIG


connection_string = connection_string = (
    f"mysql+mysqlconnector://{DB_CONFIG['user']}:{DB_CONFIG['password']}"
    f"@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)

# create connection to database
engine = create_engine(connection_string)

# test connection
try:
    with engine.connect() as connection:
        print("Connection to the database was successful.")
except Exception as e:
    print(f"Error connecting to the database: {e}")

Connection to the database was successful.


## Load CSV to RDS MySQL Database

### Load CSV data into RDS database using SQLAlchemy models
#### Apartment attributes data = apartment_attributes.csv

In [39]:
# Load apartment attributes into the database
try:
    with engine.connect() as connection:
        print("Loading apartment attributes into the database...")
        apt_att.to_sql(
            "apartment_attributes", con=connection, if_exists="replace", index=False
        )
        print("Apartment attributes loaded into the database successfully 🎉.")
except Exception as e:
    print(f"Error loading apartment data attributes into the database: {e}")

Loading apartment attributes into the database...
Apartment attributes loaded into the database successfully 🎉.


In [ ]:
-- confirm data in the apartment_attributes table
-- by counting the number of rows
SELECT COUNT(*) FROM apartment_attributes;

#### Apartments data = apartments.csv

In [ ]:
-- drop already existing tables if there is
-- DROP TABLE IF EXISTS apartments;
-- DROP TABLE IF EXISTS bookings;
-- DROP TABLE IF EXISTS user_viewings;
-- DROP TABLE IF EXISTS apartment_attributes;

In [43]:
# Load apartments data into the database
try:
    with engine.connect() as connection:
        print("Loading apartment data into the database...")
        apt.to_sql("apartments", con=connection, if_exists="replace", index=False)
        print("Apartment data loaded into the database successfully 🎉.")
except Exception as e:
    print(f"Error loading apartment data into the database: {e}")


Loading apartment data into the database...
Apartment data loaded into the database successfully 🎉.


In [ ]:
-- confirm data in the apartments table
-- by counting the number of rows
SELECT COUNT(*) FROM apartments;

#### Bookings data = bookings.csv

In [44]:
# convert all currency columns record to "USD"
bookings["currency"] = "USD"
bookings.info()
bookings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   booking_id      500000 non-null  int64  
 1   user_id         500000 non-null  int64  
 2   apartment_id    500000 non-null  int64  
 3   booking_date    500000 non-null  object 
 4   checkin_date    500000 non-null  object 
 5   checkout_date   500000 non-null  object 
 6   total_price     500000 non-null  float64
 7   currency        500000 non-null  object 
 8   booking_status  500000 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 34.3+ MB


,booking_id,user_id,apartment_id,booking_date,checkin_date,checkout_date,total_price,currency,booking_status
0,1,2920,130940,11/11/2022,07/09/2024,30/06/2023,2923.67,USD,confirmed
1,2,2788,191497,17/05/2024,07/07/2024,06/09/2021,4645.29,USD,pending
2,3,9285,139117,26/11/2022,12/02/2025,07/04/2023,1683.99,USD,canceled
3,4,8161,118131,09/07/2020,25/10/2024,08/02/2021,1870.62,USD,confirmed
4,5,6733,43470,08/05/2024,31/01/2022,29/10/2024,3379.44,USD,pending


In [45]:
# Load bookings data into the database
try:
    with engine.connect() as connection:
        print("Loading bookings into the database...")
        bookings.to_sql("bookings", con=connection, if_exists="replace", index=False)
        print("Bookings data loaded into the database successfully 🎉.")
except Exception as e:
    print(f"Error loading bookings data into the database: {e}")

Loading bookings into the database...
Bookings data loaded into the database successfully 🎉.


In [ ]:
-- confirm data in the bookings table
-- by counting the number of rows
SELECT COUNT(*) FROM bookings;
SELECT * FROM bookings LIMIT 10;

#### User viewing data = user_viewing.csv

In [46]:
# Load user viewing data into the database
try:
    with engine.connect() as connection:
        print("Loading user viewing into the database...")
        user_viewing.to_sql(
            "user_viewings", con=connection, if_exists="replace", index=False
        )
        print("User viewing data loaded into the database successfully 🎉.")
except Exception as e:
    print(f"Error loading user viewing data into the database: {e}")


Loading user viewing into the database...
User viewing data loaded into the database successfully 🎉.


In [ ]:
-- confirm data in the user_viewings table
-- by counting the number of rows
SELECT COUNT(*) FROM user_viewings;
SELECT * FROM user_viewings LIMIT 10;